In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go

# Exploration des émissions de co2

In [ ]:
emissions = pd.read_csv('../data/emission-co2-perimetre-complet.csv',sep=';')
print(emissions.shape)
emissions.head()

In [ ]:
def rename_columns(emissions_df):
    emissions_df.rename(columns={
        'Distance entre les gares':'Distance', 
        "Train - Empreinte carbone (kgCO2e)":"Train", 
        "Autocar longue distance - Empreinte carbone (kgCO2e)": "Autocar", 
        "Avion - Empreinte carbone (kgCO2e)":"Avion", 
        "Voiture électrique (2,2 pers.) - Empreinte carbone (kgCO2e)":"Voiture électrique", 
        "Voiture thermique (2,2 pers.) - Empreinte carbone (kgCO2e)":"Voiture thermique"
    }, inplace=True)
    return emissions_df

In [ ]:
emissions = rename_columns(emissions)
emissions.head()

On doit également filtrer les lignes qui sont des trains internationaux qui ne sont pas forcément sur le réseau français.

In [ ]:
print(emissions["Transporteur"].unique())

In [ ]:
def filter_transporteur(emissions_df):
    emissions_df = emissions_df.query("Transporteur != 'International'")
    return emissions_df

emissions = filter_transporteur(emissions)

In [ ]:
def generate_line_chart(emissions_df, log_scale=False):
    fig = go.Figure()
    # On ajoute les courbes pour chaque moyen de transport
    emissions_df = emissions_df.sort_values(by='Distance')
    moyens_transport = ["Autocar", "Voiture électrique", "Voiture thermique", "Avion"]
    
    # Solutionn inspirée de https://github.com/plotly/plotly.py/issues/4278 pour afficher l'origine et la destination des trajets dans le hover
    # On ajoute une courbe invisible pour chaque moyen de transport, et on change son hovertext pour afficher l'origine et la destination
    fig.add_trace(go.Scatter(
        x=emissions_df['Distance'], 
        y=emissions_df['Train'], 
        opacity=0, 
        showlegend=False,
        hovertext=emissions_df['Origine'] + ' - ' + emissions_df['Destination'],
        hovertemplate='<b>%{hovertext}</b><extra></extra>'
        ))
    
    fig.add_trace(go.Scatter(
        x=emissions_df['Distance'],
        y=emissions_df['Train'],
        mode='lines+markers',
        name='Train',
        connectgaps=True,
        hovertext=emissions_df['Transporteur'],
        hovertemplate='<b>%{hovertext}</b> : %{y:.2f} kgCO2e <extra></extra>',
    ))
    for column in moyens_transport:
        fig.add_trace(go.Scatter(
            x=emissions_df['Distance'],
            y=emissions_df[column],
            mode='lines+markers',
            name=column,
            connectgaps=True,
            hovertemplate='%{y:.2f} kgCO2e'
        ))
    
    fig.update_layout(
        title='Émissions de CO2 par moyen de transport en fonction de la distance',
        xaxis_title='Distance entre les gares (km)',
        yaxis_title='Empreinte carbone (kgCO2e)',
        legend_title='Moyen de transport',
        hovermode='x unified'
    )
    
    if log_scale:
        fig.update_yaxes(
            title='Émissions de CO2 par moyen de transport en fonction de la distance (échelle logarithmique)', 
            type="log"
    )
    
    return fig

generate_line_chart(emissions,True)    

In [ ]:
def generate_bar_graph(emissions_df):
    
    

generate_bar_graph(emissions)